In [13]:
# Importations nécessaires
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Input

In [14]:
# Configuration
base_logdir = "./logs"
seed = 42
batch_size = 1
epochs = 100
lr = 0.001
img_height, img_width = 32, 32

In [15]:
# Fixer les graines aléatoires pour la reproductibilité
np.random.seed(seed)
tf.random.set_seed(seed)

In [16]:

# Définir le nom de l'expérience
experiment_name = f"mlp_dataset_seed_{seed}_bs_{batch_size}_lr_{lr}"
final_log_dir = os.path.join(base_logdir, experiment_name)

In [17]:
# Fonction de normalisation Z-score
def z_score_normalize(data):
    mean = np.mean(data)
    std = np.std(data)
    return (data - mean) / std

In [18]:
# Fonction d'encodage des étiquettes
def encode_labels(labels):
    encoded = []
    for label in labels:
        if label == 0:  # metal
            encoded.append([1, -1, -1])
        elif label == 1:  # paper
            encoded.append([-1, 1, -1])
        else:  # plastic
            encoded.append([-1, -1, 1])
    return np.array(encoded)

In [19]:
# Fonction de chargement et prétraitement des images
def load_and_preprocess_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.cast(img, tf.float32) / 255.0
    img = z_score_normalize(img)
    return img

In [20]:
# Générateurs de données
train_datagen = ImageDataGenerator(preprocessing_function=z_score_normalize)
test_datagen = ImageDataGenerator(preprocessing_function=z_score_normalize)

train_generator = train_datagen.flow_from_directory(
    '../../dataset/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

test_generator = test_datagen.flow_from_directory(
    '../../dataset/test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

Found 30154 images belonging to 3 classes.
Found 3208 images belonging to 3 classes.


In [21]:
# Encoder les étiquettes
train_generator.classes = encode_labels(train_generator.classes)
test_generator.classes = encode_labels(test_generator.classes)


In [22]:
# Définition du modèle
model = Sequential([
    Input(shape=(img_height, img_width, 1)),
    Flatten(),
    Dense(256, activation='tanh'),
    Dense(128, activation='tanh'),
    Dense(64, activation='tanh'),
    Dense(3, activation='tanh')
])

In [23]:
# Compilation du modèle
optimizer = SGD(learning_rate=lr, momentum=0.0)
model.compile(
    optimizer=optimizer,
    loss='mean_squared_error',
    metrics=['accuracy']
)


In [25]:

# Entraînement du modèle
# Entraînement du modèle
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[TensorBoard(log_dir=final_log_dir)]
)


Epoch 1/100
   30/30154 ━━━━━━━━━━━━━━━━━━━━ 2:05 4ms/step - accuracy: 0.3433 - loss: 0.7691       

/home/adam/esgi/3BigData/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  106/30154 ━━━━━━━━━━━━━━━━━━━━ 2:03 4ms/step - accuracy: 0.4117 - loss: 0.7284

/home/adam/esgi/3BigData/.venv/lib/python3.12/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


20288/30154 ━━━━━━━━━━━━━━━━━━━━ 51s 5ms/step - accuracy: 0.4795 - loss: nan 

/tmp/ipykernel_46036/1367856415.py:5: RuntimeWarning: invalid value encountered in divide
  return (data - mean) / std


30154/30154 ━━━━━━━━━━━━━━━━━━━━ 169s 6ms/step - accuracy: 0.4496 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 2/100


2024-07-20 16:05:15.743226: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
2024-07-20 16:05:15.765975: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
# Visualisation des résultats
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:

# Sauvegarde du modèle
model.save('mlp_model.h5')